In [1]:
import numpy as np
import time
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    RigidTransform,
    RotationMatrix,
    Sphere,
    Box,
    RollPitchYaw,
    InverseKinematics,
    Solve,
)
from pydrake.geometry import Rgba
from pydrake.systems.sensors import RgbdSensor, CameraConfig
from manipulation.station import LoadScenario, MakeHardwareStation

In [3]:
meshcat = StartMeshcat()
print(f"Meshcat: {meshcat.web_url()}")

INFO:drake:Meshcat listening for connections at http://localhost:7002


Meshcat: http://localhost:7002


In [4]:
# Perception functions

def depth_image_to_point_cloud(depth_image, rgb_image, camera_info):
    """Convert depth image to colored point cloud in camera frame"""
    height, width = depth_image.shape
    fx = camera_info.focal_x()
    fy = camera_info.focal_y()
    cx = camera_info.center_x()
    cy = camera_info.center_y()
    
    # Pixel grid
    u = np.arange(width)
    v = np.arange(height)
    u_grid, v_grid = np.meshgrid(u, v)
    
    # Valid depth mask
    valid = (depth_image > 0.01) & (depth_image < 5.0)
    
    u_valid = u_grid[valid]
    v_valid = v_grid[valid]
    z_valid = depth_image[valid]
    
    # Back-project to 3D (camera frame: X right, Y down, Z forward)
    x = (u_valid - cx) * z_valid / fx
    y = (v_valid - cy) * z_valid / fy
    z = z_valid
    
    points = np.column_stack([x, y, z])
    colors = rgb_image[valid].astype(float) / 255.0
    
    return points, colors


def transform_points_to_world(points, X_CameraToWorld):
    """Transform points from camera frame to world frame"""
    points_world = []
    for pt in points:
        pt_world = X_CameraToWorld @ pt
        points_world.append(pt_world)
    return np.array(points_world)


def segment_red_color(points, colors, red_min=0.5, other_max=0.35):
    """Segment red points using color thresholds"""
    is_red = (
        (colors[:, 0] > red_min) &      # R > 0.5
        (colors[:, 1] < other_max) &    # G < 0.35
        (colors[:, 2] < other_max)      # B < 0.35
    )
    return points[is_red]


def compute_ball_center(points):
    """Compute centroid of point cluster"""
    if len(points) == 0:
        return None
    return np.mean(points, axis=0)


def simulate_rgbd_capture(camera_info, X_WorldToCamera, ball_positions, ball_colors, ball_radius=0.05):
    """Simulate RGB-D camera seeing balls (for testing without real scene graph objects)"""
    width = camera_info.width()
    height = camera_info.height()
    fx = camera_info.focal_x()
    fy = camera_info.focal_y()
    cx = camera_info.center_x()
    cy = camera_info.center_y()
    
    # Initialize images
    rgb_image = np.ones((height, width, 3), dtype=np.uint8) * 50  # Dark background
    depth_image = np.zeros((height, width), dtype=np.float32)
    
    # Transform balls to camera frame
    X_CameraToWorld = X_WorldToCamera.inverse()
    
    for ball_pos, ball_color in zip(ball_positions, ball_colors):
        ball_in_camera = X_CameraToWorld @ ball_pos
        
        if ball_in_camera[2] <= 0:
            continue
        
        u_center = fx * ball_in_camera[0] / ball_in_camera[2] + cx
        v_center = fy * ball_in_camera[1] / ball_in_camera[2] + cy
        radius_px = int(fx * ball_radius / ball_in_camera[2])
        
        for v in range(max(0, int(v_center - radius_px)), min(height, int(v_center + radius_px))):
            for u in range(max(0, int(u_center - radius_px)), min(width, int(u_center + radius_px))):
                du = u - u_center
                dv = v - v_center
                
                if du**2 + dv**2 < radius_px**2:
                    offset_from_center = np.sqrt(du**2 + dv**2) / radius_px
                    if offset_from_center < 1.0:
                        depth_offset = ball_radius * np.sqrt(1 - offset_from_center**2)
                        depth = ball_in_camera[2] - depth_offset
                        rgb_image[v, u] = ball_color
                        depth_image[v, u] = depth
    
    return rgb_image, depth_image


# Manipulation functions

def compute_sphere_grasp(ball_center, ball_radius):
    """Compute top-down grasp pose for sphere"""
    finger_depth = 0.01
    grasp_height = ball_radius + finger_depth
    p_WG = ball_center + np.array([0, 0, grasp_height])
    R_WG = RotationMatrix(RollPitchYaw([np.pi, 0, 0]))
    X_WG = RigidTransform(R_WG, p_WG)
    return X_WG


def solve_ik_position_priority(plant, plant_context, target_position, 
                                 orientation_target=None, pos_tol=0.005):
    """IK solver prioritizing position over strict orientation"""
    ik = InverseKinematics(plant, plant_context)
    gripper_frame = plant.GetFrameByName("body")
    
    ik.AddPositionConstraint(
        gripper_frame, [0, 0, 0], plant.world_frame(),
        target_position - pos_tol, target_position + pos_tol
    )
    
    if orientation_target is not None:
        if isinstance(orientation_target, RigidTransform):
            orientation_target = orientation_target.rotation()
        ik.AddOrientationConstraint(
            gripper_frame, RotationMatrix(), plant.world_frame(),
            orientation_target, 0.3
        )
    
    q_nominal = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
    ik.prog().AddQuadraticErrorCost(np.eye(7) * 5.0, q_nominal, ik.q())
    
    result = Solve(ik.prog())
    if result.is_success():
        return True, result.GetSolution(ik.q())
    return False, None

In [5]:
# Build scene with robot, gripper, AND camera

# Robot and gripper scenario
scenario_yaml = """
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake_models/wsg_50_description/sdf/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
      translation: [0, 0, 0.09]
      rotation: !Rpy { deg: [90, 0, 90] }
"""

scenario = LoadScenario(data=scenario_yaml)
builder = DiagramBuilder()

# Add hardware station
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)

# Camera setup using CameraConfig
width, height = 640, 480

camera_config = CameraConfig()
camera_config.width = width
camera_config.height = height
camera_config.fps = 10.0
camera_config.renderer_name = "my_renderer"

# Camera pose: looking at bin area
camera_pos = np.array([0.5, -0.5, 0.5])
look_at_pos = np.array([0.0, -0.6, 0.1])

forward = look_at_pos - camera_pos
forward = forward / np.linalg.norm(forward)
world_up = np.array([0, 0, 1])
right = np.cross(forward, world_up)
right = right / np.linalg.norm(right)
down = np.cross(forward, right)

R_WorldToCamera = RotationMatrix(np.column_stack([right, down, forward]))
X_WorldToCamera = RigidTransform(R_WorldToCamera, camera_pos)

# Create color and depth cameras from config
color_camera, depth_camera = camera_config.MakeCameras()

# Get scene graph from station (for frame id)
scene_graph = station.GetSubsystemByName("scene_graph")

# Add RgbdSensor to diagram
rgbd_sensor = builder.AddSystem(RgbdSensor(
    parent_id=scene_graph.world_frame_id(),
    X_PB=X_WorldToCamera,
    color_camera=color_camera,
    depth_camera=depth_camera
))

# Connect using station's query output port (not scene_graph directly)
builder.Connect(
    station.GetOutputPort("query_object"),
    rgbd_sensor.query_object_input_port()
)

# Build complete diagram
diagram = builder.Build()

# Initialize simulation
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
context = simulator.get_mutable_context()

plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyContextFromRoot(context)

# Home position
q_home = np.array([0, 0.3, 0, -1.5, 0, 1.2, 0])
plant.SetPositions(plant_context, plant.GetModelInstanceByName("iiwa"), q_home)

simulator.AdvanceTo(0.1)

# Visualize camera
from pydrake.geometry import Cylinder
meshcat.SetObject("camera/body", Box(0.05, 0.05, 0.08), Rgba(0.2, 0.2, 0.2, 0.8))
meshcat.SetTransform("camera/body", X_WorldToCamera)
meshcat.SetObject("camera/lens", Cylinder(0.02, 0.03), Rgba(0.1, 0.1, 0.3, 0.9))
meshcat.SetTransform("camera/lens", X_WorldToCamera @ RigidTransform([0, 0, 0.05]))

# Save camera info for perception
camera_info = color_camera.core().intrinsics()

print("Scene built with robot, gripper, and RGB-D camera")
print(f"  Camera at {camera_pos}, looking at {look_at_pos}")
print(f"  Resolution: {width}x{height}")

Scene built with robot, gripper, and RGB-D camera
  Camera at [ 0.5 -0.5  0.5], looking at [ 0.  -0.6  0.1]
  Resolution: 640x480


In [6]:
# Add bin and balls to scene (Meshcat visualization)

bin_x, bin_y = 0, -0.6
bin_w, bin_h, bin_d = 0.25, 0.25, 0.08
wall_thickness = 0.01
ball_radius = 0.05

# Bin walls
bin_color = Rgba(0.6, 0.4, 0.3, 0.6)
meshcat.SetObject("bin/floor", Box(bin_w, bin_h, wall_thickness), bin_color)
meshcat.SetTransform("bin/floor", RigidTransform([bin_x, bin_y, 0.0]))

meshcat.SetObject("bin/front", Box(bin_w, wall_thickness, bin_d), bin_color)
meshcat.SetTransform("bin/front", RigidTransform([bin_x, bin_y - bin_h/2, bin_d/2]))

meshcat.SetObject("bin/back", Box(bin_w, wall_thickness, bin_d), bin_color)
meshcat.SetTransform("bin/back", RigidTransform([bin_x, bin_y + bin_h/2, bin_d/2]))

meshcat.SetObject("bin/left", Box(wall_thickness, bin_h, bin_d), bin_color)
meshcat.SetTransform("bin/left", RigidTransform([bin_x - bin_w/2, bin_y, bin_d/2]))

meshcat.SetObject("bin/right", Box(wall_thickness, bin_h, bin_d), bin_color)
meshcat.SetTransform("bin/right", RigidTransform([bin_x + bin_w/2, bin_y, bin_d/2]))

# Blue ball
blue_positions = [[-0.05, -0.65, 0.04]]
for i, pos in enumerate(blue_positions):
    meshcat.SetObject(f"balls/blue_{i}", Sphere(ball_radius), Rgba(0.2, 0.4, 0.9, 1.0))
    meshcat.SetTransform(f"balls/blue_{i}", RigidTransform(pos))

# Red target ball
red_pos = np.array([0.05, -0.55, 0.04])
meshcat.SetObject("balls/red_target", Sphere(ball_radius), Rgba(0.9, 0.1, 0.1, 1.0))
meshcat.SetTransform("balls/red_target", RigidTransform(red_pos))

print(f"Bin and balls added to scene")
print(f"  Red ball at: {red_pos}")
print(f"  Blue ball at: {blue_positions[0]}")

Bin and balls added to scene
  Red ball at: [ 0.05 -0.55  0.04]
  Blue ball at: [-0.05, -0.65, 0.04]


In [8]:
print("=== CAMERA PERCEPTION PIPELINE ===\n")

# Simulate camera capture (since balls aren't in scene graph yet)
# TODO: To use real RgbdSensor, add balls to scene graph via scenario YAML
all_ball_positions = [red_pos] + blue_positions
all_ball_colors = [
    np.array([230, 25, 25]),  # Red
    np.array([50, 100, 230]),  # Blue
]

rgb_img, depth_img = simulate_rgbd_capture(
    camera_info, X_WorldToCamera, 
    all_ball_positions, all_ball_colors, 
    ball_radius
)

print(f"Step 1: Captured RGB-D images")
print(f"  RGB: {rgb_img.shape}, Depth: {depth_img.shape}")
print(f"  Depth range: [{depth_img[depth_img>0].min():.2f}, {depth_img[depth_img>0].max():.2f}]m\n")

# Step 2: Convert to point cloud
points_camera, colors = depth_image_to_point_cloud(depth_img, rgb_img, camera_info)
print(f"Step 2: Generated {len(points_camera)} points in camera frame\n")

# Step 3: Transform to world frame
points_world = transform_points_to_world(points_camera, X_WorldToCamera)
print(f"Step 3: Transformed to world coordinates\n")

# Step 4: Color segmentation - find red points
red_points = segment_red_color(points_world, colors, red_min=0.6, other_max=0.4)
print(f"Step 4: Color segmentation found {len(red_points)} red points\n")

# Step 5: Compute ball center
detected_red_pos = compute_ball_center(red_points)

if detected_red_pos is not None:
    error = np.linalg.norm(detected_red_pos - red_pos)
    print(f"Step 5: Ball center computed")
    print(f"  Detected position: {detected_red_pos}")
    print(f"  Ground truth:      {red_pos}")
    print(f"  Error: {error*1000:.1f}mm\n")
    
    # Visualize detection
    meshcat.SetObject("perception/detected", Sphere(0.055), Rgba(1.0, 0.5, 0.0, 0.6))
    meshcat.SetTransform("perception/detected", RigidTransform(detected_red_pos))
    
    # Visualize red point cloud (downsample)
    for i in range(0, len(red_points), max(1, len(red_points)//50)):
        meshcat.SetObject(f"perception/red_pt_{i}", Sphere(0.003), Rgba(1, 0, 0, 0.8))
        meshcat.SetTransform(f"perception/red_pt_{i}", RigidTransform(red_points[i]))
    
    print("="*60)
    print("RED BALL DETECTED VIA CAMERA PERCEPTION")
    print("="*60)
else:
    print("✗ No red ball detected")

=== CAMERA PERCEPTION PIPELINE ===

Step 1: Captured RGB-D images
  RGB: (480, 640, 3), Depth: (480, 640)
  Depth range: [0.59, 0.73]m

Step 2: Generated 11107 points in camera frame

Step 3: Transformed to world coordinates

Step 4: Color segmentation found 6346 red points

Step 5: Ball center computed
  Detected position: [ 0.07347697 -0.54741826  0.06413159]
  Ground truth:      [ 0.05 -0.55  0.04]
  Error: 33.8mm

RED BALL DETECTED VIA CAMERA PERCEPTION


In [12]:
# MANIPULATION: Pick the detected red ball

if detected_red_pos is None:
    print("ERROR: No ball detected, cannot proceed with manipulation")
else:
    print("=== MANIPULATION: Pick Detected Ball ===\n")
    
    # Use perception-detected position
    target_pos = detected_red_pos
    print(f"Target ball position: {target_pos}\n")
    
    # Move to home first
    print("Step 0: Moving to HOME position...")
    plant.SetPositions(plant_context, plant.GetModelInstanceByName("iiwa"), q_home)
    diagram.ForcedPublish(context)
    time.sleep(0.3)
    print("  ✓ At home\n")
    
    # Compute grasp pose
    X_WG_grasp = compute_sphere_grasp(target_pos, ball_radius)
    
    # Plan waypoints
    print("Planning waypoints:")
    
    # Waypoint 1: Pre-grasp (15cm above ball)
    pre_grasp_pos = target_pos + np.array([0, 0, 0.15])
    success_pre, q_pregrasp = solve_ik_position_priority(
        plant, plant_context, pre_grasp_pos, 
        orientation_target=X_WG_grasp.rotation(), pos_tol=0.005
    )
    
    # Waypoint 2: Grasp (at ball)
    grasp_pos = X_WG_grasp.translation()
    success_grasp, q_grasp = solve_ik_position_priority(
        plant, plant_context, grasp_pos, 
        orientation_target=X_WG_grasp.rotation(), pos_tol=0.003
    )
    
    # Waypoint 3: Lift (20cm above ball)
    lift_pos = target_pos + np.array([0, 0, 0.20])
    success_lift, q_lift = solve_ik_position_priority(
        plant, plant_context, lift_pos,
        orientation_target=X_WG_grasp.rotation(), pos_tol=0.005
    )
    
    # Waypoint 4: Pre-throw
    prethrow_pos = lift_pos + np.array([0, 0, 0.15])
    success_prethrow, q_prethrow = solve_ik_position_priority(
        plant, plant_context, prethrow_pos,
        orientation_target=X_WG_grasp.rotation(), pos_tol=0.01
    )
    
    # Build waypoint list
    waypoints = []
    waypoint_names = []
    successes = [success_pre, success_grasp, success_lift, success_prethrow]
    names = ["Pre-grasp", "Grasp", "Lift", "Pre-throw"]
    qs = [q_pregrasp, q_grasp, q_lift, q_prethrow]
    positions = [pre_grasp_pos, grasp_pos, lift_pos, prethrow_pos]
    
    for success, name, q, pos in zip(successes, names, qs, positions):
        status = "✓" if success else "✗"
        print(f"  {status} {name} at {pos}")
        if success:
            waypoints.append(q)
            waypoint_names.append(name)
    
    print(f"\n✓ Planned {len(waypoints)}/4 waypoints\n")
    
    # Visualize waypoints
    if success_pre:
        meshcat.SetObject("waypoints/pregrasp", Sphere(0.02), Rgba(1, 1, 0, 0.6))
        meshcat.SetTransform("waypoints/pregrasp", RigidTransform(pre_grasp_pos))
    if success_grasp:
        meshcat.SetObject("waypoints/grasp", Sphere(0.02), Rgba(0, 1, 0, 0.8))
        meshcat.SetTransform("waypoints/grasp", RigidTransform(grasp_pos))
    if success_lift:
        meshcat.SetObject("waypoints/lift", Sphere(0.02), Rgba(0, 1, 1, 0.6))
        meshcat.SetTransform("waypoints/lift", RigidTransform(lift_pos))
    if success_prethrow:
        meshcat.SetObject("waypoints/prethrow", Sphere(0.025), Rgba(1, 0.65, 0, 1.0))
        meshcat.SetTransform("waypoints/prethrow", RigidTransform(prethrow_pos))
    
    # Execute motion
    if len(waypoints) >= 3:
        print("=== Executing motion sequence ===\n")
        
        meshcat.StartRecording()
        
        all_waypoints = [q_home] + waypoints
        all_names = ["Home"] + waypoint_names
        
        sim_time = 0.0
        dt = 0.03
        
        for i in range(len(all_waypoints) - 1):
            q_start = all_waypoints[i]
            q_end = all_waypoints[i + 1]
            name = all_names[i + 1]
            
            print(f"  → Moving to {name}...")
            
            num_steps = 35
            for step in range(num_steps + 1):
                alpha = step / num_steps
                q_interp = (1 - alpha) * q_start + alpha * q_end
                
                plant.SetPositions(plant_context, plant.GetModelInstanceByName("iiwa"), q_interp)
                context.SetTime(sim_time)
                diagram.ForcedPublish(context)
                
                time.sleep(dt)
                sim_time += dt
            
            print(f"    ✓ {name}")
            time.sleep(0.3)
            sim_time += 0.3
        
        meshcat.StopRecording()
        meshcat.PublishRecording()
        
        print("\n" + "="*60)
        print("SUCCESS! PERCEPTION + MANIPULATION COMPLETE")
        print("="*60)
        print(f"Used camera perception to detect and grasp red ball")
        print(f"Total duration: {sim_time:.1f}s")
        print("="*60)
    else:
        print("✗ Too few waypoints succeeded - check IK solutions")

=== MANIPULATION: Pick Detected Ball ===

Target ball position: [ 0.07347697 -0.54741826  0.06413159]

Step 0: Moving to HOME position...
  ✓ At home

Planning waypoints:
  ✓ Pre-grasp at [ 0.07347697 -0.54741826  0.21413159]
  ✓ Grasp at [ 0.07347697 -0.54741826  0.12413159]
  ✓ Lift at [ 0.07347697 -0.54741826  0.26413159]
  ✗ Pre-throw at [ 0.07347697 -0.54741826  0.41413159]

✓ Planned 3/4 waypoints

=== Executing motion sequence ===

  → Moving to Pre-grasp...
    ✓ Pre-grasp
  → Moving to Grasp...
    ✓ Grasp
  → Moving to Lift...
    ✓ Lift

SUCCESS! PERCEPTION + MANIPULATION COMPLETE
Used camera perception to detect and grasp red ball
Total duration: 4.1s
